In [1]:
from myBdd import *

In [2]:
f = boolfunc.Expression(
    r"input\1.pcn")
print(f)


4
3
3 1 2 3
2 -2 4
2 -3 4


In [3]:
ordering = [1,4,2,3]
print(ordering)
a = BDD(f, ordering)
a

[1, 4, 2, 3]


In [5]:
for node in a.dfsPreorder():
    print(node.exp)

4
3
3 1 2 3
2 -2 4
2 -3 4
4
2
2 -3 4
2 -2 4
4
0
4
2
1 -3
1 -2
4
1
0
4
1
1 -3
4
3
2 2 3
2 -3 4
2 -2 4
4
1
2 2 3
4
1
1 3
4
3
2 2 3
1 -3
1 -2
